In [ ]:
from datasets import load_dataset
import torch
from transformers import BertTokenizerFast, BertModel, BertConfig
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import glob
import os

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', use_lower_case=True)
config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
model = BertModel.from_pretrained('bert-base-uncased', config=config)

In [ ]:
dataset = load_dataset('imdb', split='train')

In [ ]:
if(os.path.exists("./data")):
    files = glob.glob('./data/*')
    for f in files:
        os.remove(f)
else:
    os.makedirs("./data")

In [ ]:
batch_text = []
dictionary_list = []
count = 0

for i in tqdm(range(len(dataset))):
#     if(i<=500):
        if(i%4==0):  #batch size can be increased depending on your RAM
            batch_text.append(dataset["text"][i])

            encoding = tokenizer(batch_text, return_tensors='pt', padding='max_length', truncation=True, max_length=256)
            input_ids = encoding['input_ids']
            attention_mask = encoding['attention_mask']
            outputs = model(input_ids, attention_mask)
            hidden_states = outputs[2][1:]

            batch_text = []

            for q in range(len(hidden_states[0])):
                for j in range(0,12):
                    new_row = {'embeddings':hidden_states[j][q][0].cpu().detach().numpy(), 'layers': j+1}
                    dictionary_list.append(new_row)
            
            dictionary_list = np.save(f"./data/batch_{count}", dictionary_list, allow_pickle=True)
            dictionary_list = []
            count += 1

        else:
            batch_text.append(dataset["text"][i])

In [ ]:
dictionary_list = []

files = glob.glob("./data/*.npy")

for j in range(len(files)):
    alpha = np.load(f"./data/batch_{j}.npy", allow_pickle = True)
    for i in range(len(alpha)):
        new_row = {'embeddings':alpha[i]["embeddings"], 'layers': alpha[i]["layers"]}
        dictionary_list.append(new_row)

In [ ]:
df = pd.DataFrame.from_dict(dictionary_list)
df.to_pickle("./imdb.pkl")

In [ ]:
df.layers.unique()

### Changes incase of converting from 12 layers to Top, Middle, and Bottom layers